# Light sources

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm
import torch.optim as optim

# Idea: Rendering of light sources: render a bit into the negative t with different color?
# + render the source 'outline' as a line/disk/surface

optics = nn.Sequential(
    tlm.Gap(10),
    tlm.Rotate(
        tlm.PointSourceAtInfinity(beam_diameter=18.5),
        angles = (-15, -5),
    ),
    tlm.Gap(10),
    tlm.PointSourceAtInfinity(beam_diameter=18.5),
)

tlm.show(optics, mode="2D", end=40)
tlm.show(optics, mode="3D", end=40, base=1000)

In [ ]:
optics = nn.Sequential(
    tlm.PointSource(45),
)

tlm.show(optics, mode="2D", end=10)